In [ ]:
import os
import wget
import datetime
from time import sleep, time
from pytube import YouTube, Playlist
from moviepy.editor import ffmpeg_tools as ff
import eyed3
from eyed3.id3.frames import ImageFrame
from rich.progress import track

# Todo
files = List all file names
For file in files:
    artist = file.couper avant le '-'
    artist = artist.enlever l'espace avant et après
    mutagen tag artist

In [ ]:
def download_pl(short_pl, n_mus, output):
    """
    Select the correct streams (Adaptive/DASH, audio, .mp4) and download the file
    :param short_pl: (list) List of URLs of only the selected YouTube videos
    :param n_mus: (int) number of musics we want to download
    :param output: (str) String of the output path where the musics are to be downloaded
    :return: n_music: (int) number of musics we want to download
    """
    t1 = time()
    print('\nTagging ...', end='\n\n')

    for i, url in enumerate(track(short_pl, description='[red]Tagging ... ')):

        mp4_file = f'.{title}.mp4'
        mp3_file = f'{title}.mp3'

        # Fetch all the streams in Dash/adaptive, audio and .mp4 format
        audios_dash_mp4 = mus.streams.filter(adaptive=True,
                                             only_audio=True,
                                             file_extension='mp4')

        # Download the best stream to .mp4
        best_audio = audios_dash_mp4[-1]  # The last stream has the best quality
        print(f'🌟 {i+1}: {title}')
        best_audio.download(output_path=output,
                            filename=mp4_file,
                            max_retries=3)

        # Convert .mp4 to .mp3 and clean the .mp4
        ff.ffmpeg_extract_audio(os.path.join(output, mp4_file), os.path.join(output, mp3_file))
        os.remove(os.path.join(output, mp4_file))

        # Download the thumbnail
        pic_path_name = wget.download(mus.thumbnail_url, output)

        # Mutagen.eyed3, to merge the thumbnail to the .mp3
        # https://stackoverflow.com/questions/38510694/how-to-add-album-art-to-mp3-file-using-python-3
        music = eyed3.load(os.path.join(output, mp3_file))
        if music.tag is None:
            music.initTag()

        # music.tag.title = 'your_title'  # Set the title
        # music.tag.album = 'your_album_name'  # Set the album name
        music.tag.images.set(3,
                             open(pic_path_name, 'rb').read(),
                             'image/jpeg')

        # Save the tags
        music.tag.save(version=eyed3.id3.ID3_V2_3)
        music.tag.save()

        # Clean the thumbnail file
        os.remove(pic_path_name)
        print('\n')

    # Compute the time spent on the download part
    t2 = time()
    time_spent_sec = t2 - t1
    time_spent_min = time_spent_sec / 60

    print('\n\n')
    print(f'{n_mus} musics downloaded in {"%.2f"%time_spent_sec}s ({"%.2f"%time_spent_min}mins) ! ✅')